# Training stocks model

Steps
* [Create feature vector](#Create-feature-vector)
* [Train locally using mlrun pytorch framework](#Train-locally-using-mlrun-pytorch-framework)

In [1]:
# !pip install -U torch

In [2]:
import mlrun
project = mlrun.get_or_create_project(name='stocks',user_project=True, context="src/")

> 2022-09-04 13:56:36,105 [info] loaded project stocks from MLRun DB


## Create feature vector

In [3]:
# Define the list of features we will be using
features = ['stocks.*',
            'news.sentiment',
            ]

# Import MLRun's Feature Store
import mlrun.feature_store as fstore

# Define the feature vector name for future reference
fv_name = 'stocks'

# Define the feature vector using our Feature Store (fstore)
transactions_fv = fstore.FeatureVector(fv_name, 
                          features, 
                          description='stocks information')

# Save the feature vector in the Feature Store
transactions_fv.save()

In [4]:
# Get offline feature vector as dataframe and save the dataset to parquet
import datetime
start_time = datetime.datetime.now()-datetime.timedelta(59)
end_time = datetime.datetime.now()-datetime.timedelta(0)
train_dataset = fstore.get_offline_features(fv_name,start_time=start_time,end_time=end_time, entity_timestamp_column = 'Datetime')
#train_dataset = fstore.get_offline_features(fv_name,with_indexes=True, entity_timestamp_column = 'Datetime')
df = train_dataset.to_dataframe()
df

,Open,High,Low,Close,Volume,ticker2onehot_A,ticker2onehot_AAL,ticker2onehot_AAP,ticker2onehot_AAPL,ticker2onehot_ABBV,ticker2onehot_ABC,ticker2onehot_ABMD,ticker2onehot_ABT,ticker2onehot_ACN,ticker2onehot_ADBE,sentiment
0,13.985000,14.110000,13.985000,14.040600,0,0,1,0,0,0,0,0,0,0,0,NaN
1,121.900002,122.345001,121.900002,122.139999,0,1,0,0,0,0,0,0,0,0,0,NaN
2,108.779999,109.080002,108.690002,109.019997,0,0,0,0,0,0,0,0,1,0,0,NaN
3,140.710007,141.210007,140.580002,141.210007,0,0,0,0,0,0,1,0,0,0,0,NaN
4,150.871002,151.559998,150.580002,151.410004,0,0,0,0,0,1,0,0,0,0,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31096,128.009995,128.009995,128.009995,128.009995,0,1,0,0,0,0,0,0,0,0,0,1.0
31097,147.710007,147.710007,147.710007,147.710007,0,0,0,0,0,0,1,0,0,0,0,0.0
31098,261.239990,261.239990,261.239990,261.239990,0,0,0,0,0,0,0,1,0,0,0,1.0
31099,171.729996,171.729996,171.729996,171.729996,0,0,0,1,0,0,0,0,0,0,0,0.5


## Train locally using mlrun pytorch framework 

In [5]:
fn = mlrun.code_to_function('train_stocks_model', kind='job',image='mlrun/ml-models',handler='handler', filename='src/train_stocks.py').apply(mlrun.auto_mount())

In [6]:
import os

params = {'hidden_dim':2,
          'n_layers':1,
          'epochs':3, 
          'vector_name':'stocks',
          'seq_size':5,
          'start_time':59,
          'end_time':0,
          'batch_size':1,
          'model_filepath':os.path.join(os.getcwd(),'src')}

fn.run(local=True,watch=True, params = params)

> 2022-09-04 13:56:38,918 [info] starting run train-stocks-model-handler uid=18f025457164430c9671eb7720555513 DB=http://mlrun-api:8080
Epoch 1/3:
Training: 100% |██████████| 31041/31041 [01:01<00:00, 503.98Batch/s, MSELoss=0.000839, accuracy=0.971]
Validating: 100% |██████████| 31041/31041 [00:18<00:00, 1650.69Batch/s, MSELoss=0.00367, accuracy=0.939] 

Summary:
+----------+----------------------+
| Metrics  |        Values        |
+----------+----------------------+
| MSELoss  | 0.007823134772479534 |
| accuracy |  0.8919357061386108  |
+----------+----------------------+

Epoch 2/3:
Training: 100% |██████████| 31041/31041 [01:01<00:00, 507.59Batch/s, MSELoss=0.000107, accuracy=0.99] 
Validating: 100% |██████████| 31041/31041 [00:18<00:00, 1663.05Batch/s, MSELoss=0.000564, accuracy=0.976]

Summary:
+----------+-----------------------+
| Metrics  |        Values         |
+----------+-----------------------+
| MSELoss  | 0.0008675841963849962 |
| accuracy |  0.9792459607124329   |
+--

project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
stocks-dani,...20555513,0,Sep 04 13:56:39,completed,train-stocks-model-handler,v3io_user=danikind=owner=danihost=jupyter-dani-7dc4b6c678-5k58j,,hidden_dim=2n_layers=1epochs=3vector_name=stocksseq_size=5start_time=59end_time=0batch_size=1model_filepath=/User/test/demos/stocks-prediction/src,hidden_dim=2n_layers=1epochs=3vector_name=stocksseq_size=5start_time=59end_time=0batch_size=1model_filepath=/User/test/demos/stocks-prediction/srclr=0.0001training_MSELoss=0.0013903690269216895training_accuracy=0.9627123475074768validation_MSELoss=0.00019446699297986925validation_accuracy=0.9922555834054947,training_MSELoss.htmltraining_accuracy.htmlvalidation_MSELoss.htmlvalidation_accuracy.htmlMSELoss_summary.htmlaccuracy_summary.htmllr_values.htmlstocks_model_custom_objects_map.jsonstocks_model_custom_objects.zipmodel


> 2022-09-04 14:03:09,124 [info] run executed, status=completed
